In [3]:
import sys
import os
import importlib
import time
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer #计算tfidf
from sklearn.feature_extraction.text import CountVectorizer  #计算df
from sklearn.feature_extraction.text import TfidfVectorizer  #“一步到位”
from sklearn import svm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib # to save
from sklearn import metrics

id_num = []
word_article = []
words_article = []
label = []
X_train = []
y_train = []
X_test = []
y_test = []

load_start = time.time()
importlib.reload(sys)
with open('train_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()[1:]
    # random.shuffle(lines) #注意用法
    for line in lines[1:5000]:
        line = line.split(',')
        X_train.append(line[2])
        y_train.append(line[3])
    for line in lines[100000:]: #划分出测试集
        line = line.split(',')
        X_test.append(line[2])
        y_test.append(line[3])
print("Time consumption on loading:",time.time() - load_start)

train_start = time.time()
vect = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9, use_idf=1, smooth_idf=1, sublinear_tf=1)
vect.fit(X_train) # 或者直接X_train = vect.fit_transform(X_train)
X_train = vect.transform(X_train)
X_train = X_train.toarray() #由稀疏矩阵变成ndarray类型的完整矩阵。。？
pca = PCA(n_components = 18)
# X_train = pca.fit_transform(X_train)
print(X_train.shape)
print("Time consumption on training1:",time.time() - train_start)

lda = LinearDiscriminantAnalysis(n_components=None)
lda.fit(X_train, y_train)
X_train = lda.transform(X_train)
print(X_train.shape)

# clf = svm.LinearSVC(C=0.5)
# clf = svm.SVC(kernel='linear', C=1, )
# clf = SGDClassifier(loss="modified_huber", )#penalty=”elasticnet”: L2和L1的convex组合; (1 - l1_ratio) * L2 + l1_ratio * L1
# clf.fit(X_train, y_train)
scores = cross_val_score(lda, X_train, y_train , cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
# joblib.dump(clf, 'lineSVM_20000__.m')
print("Time consumption on training1+2:",time.time() - train_start)

test_start = time.time()
# clf = joblib.load('lineSVM_20000__.m')
prediction = lda.predict(X_train)
print(metrics.classification_report(y_train,prediction))
X_test = vect.transform(X_test)
X_test = X_test.toarray()
X_test = pca.fit_transform(X_test)
print(X_test.shape)
prediction = lda.predict(X_test)
print(metrics.classification_report(y_test,prediction))
# print(metrics.confusion_matrix(y_test,prediction))
print("Time consumption on testing:",time.time() - test_start)


Time consumption on loading: 1.6925623416900635
(4999, 190073)
Time consumption on training1: 23.50469422340393


/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


(4999, 18)
[ 0.99801193  0.99302094  0.99100899  0.99099099  0.99292929]
Accuracy: 0.99 (+/- 0.01)
Time consumption on training1+2: 215.6636574268341


ValueError: X has 18 features per sample; expecting 190073